# BDD name_basics

In [1]:
# Importation des modules
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re

# # Définir les options d'affichage pour afficher toutes les lignes sans troncature
# pd.set_option("display.max_rows",None)
# pd.set_option("display.max_colwidth",None)

# Modifier les options d'affichage pour limiter les décimales
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
# Téléchargement de la BDD
name_basic = pd.read_csv("data/name.basics.tsv", sep="\t")

#Affichage des premières lignes
print(name_basic.info())
name_basic.head()
# Affichage des dimensions
print("Les dimensions du dataframe sont",name_basic.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13578126 entries, 0 to 13578125
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   nconst             object
 1   primaryName        object
 2   birthYear          object
 3   deathYear          object
 4   primaryProfession  object
 5   knownForTitles     object
dtypes: object(6)
memory usage: 621.6+ MB
None
Les dimensions du dataframe sont (13578126, 6)


In [11]:
# Compter le nombre de modalités des variables catégorielles
mes_colonnes = ['nconst', 'primaryName', 'primaryProfession', 'knownForTitles']

# Dictionnaire pour stocker les résultats
resultats = {}

# Appliquer value_counts(normalize=True) * 100 pour chaque colonne
for i in mes_colonnes:
    resultats[i] = name_basic[i].value_counts(normalize=True) * 100

# Afficher les résultats
for i, valeurs in resultats.items():
    print(f"Pourcentage de chaque modalité pour la colonne '{i}':")
    print(valeurs)
    print()  # Séparateur pour plus de lisibilité

Pourcentage de chaque modalité pour la colonne 'nconst':
nconst
nm0000001    0.00
nm4372548    0.00
nm4372533    0.00
nm4372536    0.00
nm4372537    0.00
             ... 
nm13831536   0.00
nm13831537   0.00
nm13831538   0.00
nm13831539   0.00
nm9993719    0.00
Name: proportion, Length: 13578126, dtype: float64

Pourcentage de chaque modalité pour la colonne 'primaryName':
primaryName
Alex                 0.00
Michael Smith        0.00
Chris                0.00
Michael              0.00
David                0.00
                     ... 
Charles Foinette     0.00
Véronique Moulaert   0.00
Marta Staglianò      0.00
Fred Jensen III      0.00
Aayush Nair          0.00
Name: proportion, Length: 10453111, dtype: float64

Pourcentage de chaque modalité pour la colonne 'primaryProfession':
primaryProfession
\N                                                        19.24
actor                                                     17.80
actress                                                   11

#### Création du dataframe nameBasic_clean

In [5]:
# Création du dataframe nameBasic_clean
nameBasic_clean = pd.DataFrame(name_basic)

In [6]:
# Affichage des noms de l'ensemble des colonnes
print(nameBasic_clean.columns)

Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')


In [7]:
# Eliminer les colonnes birthYear and deathYear
nameBasic_clean = name_basic.drop(['birthYear','deathYear'], axis = 1)

In [8]:
# Modification des noms de colonnes
    # Création du dictionnaire associant les anciens noms aux nouveaux noms de colonnes
dictionnaire_colonne = {
    "nconst": "ID_name",
    "primaryName": "actor_name",
    "primaryProfession": "profession",
    "knownForTitles": "ID_title"
}

    # On renomme les variables grâce à la méthode rename
nameBasic_clean = nameBasic_clean.rename(dictionnaire_colonne, axis = 1)

nameBasic_clean.sample(2)

,ID_name,actor_name,profession,ID_title
1090153,nm1018225,Commander Johnny G.,\N,tt0080646
9978900,nm5428907,Geoff Ko,\N,tt2583184


In [9]:
# Transformer les valeurs \N en NaN
nameBasic_clean = nameBasic_clean.replace("\\N", np.nan)

In [10]:
# Initialisation des comptes de NaN
nan_counts_per_column = {}
total_nan_counts = 0

# Boucle sur chaque colonne et comptage des NaN
for column in nameBasic_clean.columns:
    nan_count = nameBasic_clean[column].isna().sum()
    nan_counts_per_column[column] = nan_count
    total_nan_counts += nan_count

# Affichage des résultats
print("Nombre de NaN par colonne:")
for column, count in nan_counts_per_column.items():
    print(f"Colonne {column}: {count} NaN")

print("\nNombre total de NaN dans tout le DataFrame:")
print(total_nan_counts)

Nombre de NaN par colonne:
Colonne ID_name: 0 NaN
Colonne actor_name: 56 NaN
Colonne profession: 2612196 NaN
Colonne ID_title: 1502159 NaN

Nombre total de NaN dans tout le DataFrame:
4114411


In [14]:
# Éliminer les lignes dont la colonne 'ID_title' ont la valeur NaN
nameBasic_clean = nameBasic_clean.dropna(subset=['ID_title'])

In [15]:
nameBasic_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12075967 entries, 0 to 13578124
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   ID_name     object
 1   actor_name  object
 2   profession  object
 3   ID_title    object
dtypes: object(4)
memory usage: 460.7+ MB


#### Vérification des doublons

In [18]:
# Initialisation d'un dictionnaire pour stocker le nombre de doublons par colonne
duplicate_counts = {}

# Pour chaque colonne du DataFrame 
for col in nameBasic_clean.columns:
    # Compter les occurrences de chaque valeur 
    value_counts = nameBasic_clean[col].value_counts()

    # Filtrer pour ne garder que les valeurs apparaissant plus d'une fois (doublons)
    duplicates = value_counts[value_counts > 1]

    # Calculer le nombre total de doublons en sommant les occurrences excédentaires
    num_duplicates = (duplicates - 1).sum()  # Soustraire 1 car chaque valeur unique compte déjà pour 1

    duplicate_counts[col] = num_duplicates

# Affichage du nombre de doublons par colonne
for col, count in duplicate_counts.items():
    print(f"Colonne '{col}' a {count} doublons")


Colonne 'ID_name' a 0 doublons
Colonne 'actor_name' a 2630756 doublons
Colonne 'profession' a 10567591 doublons
Colonne 'ID_title' a 6401734 doublons


### Détection des valeurs manquantes

In [13]:
# COLONNE
    # On détecte les COLONNES contenant au moins une valeur manquante
print(nameBasic_clean.isna().any(axis = 0).sum())
print()
    # on affiche les colonnes avec True ou false pour les NaN
colonne_NA = nameBasic_clean.isna().any(axis = 0)
print(colonne_NA)

3

ID_name       False
actor_name     True
profession     True
ID_title       True
dtype: bool


In [19]:
# Somme des valeurs manquantes par colonne
missing_values_per_column = nameBasic_clean.isnull().sum()
missing_values_per_column

ID_name             0
actor_name          6
profession    1485735
ID_title            0
dtype: int64

In [20]:
# Supprimer les lignes qui ont des valeurs manquantes dans 'actor_name'
nameBasic_clean = nameBasic_clean.dropna(subset=['actor_name'])
# Print the first 5 rows
print(nameBasic_clean.head().to_markdown(index=False, numalign="left", stralign="left"))

| ID_name   | actor_name      | profession                         | ID_title                                |
|:----------|:----------------|:-----------------------------------|:----------------------------------------|
| nm0000001 | Fred Astaire    | actor,miscellaneous,producer       | tt0072308,tt0050419,tt0053137,tt0027125 |
| nm0000002 | Lauren Bacall   | actress,soundtrack,archive_footage | tt0037382,tt0075213,tt0117057,tt0038355 |
| nm0000003 | Brigitte Bardot | actress,music_department,producer  | tt0057345,tt0049189,tt0056404,tt0054452 |
| nm0000004 | John Belushi    | actor,writer,music_department      | tt0072562,tt0077975,tt0080455,tt0078723 |
| nm0000005 | Ingmar Bergman  | writer,director,actor              | tt0050986,tt0083922,tt0050976,tt0069467 |


### 1-Sauvegarder en format CSV

In [21]:
# Une sauvegarde du df avec les valeurs manquantes de la colonne profession non supprimées.
nameBasic_clean.to_csv('nameBasic_clean_profession.csv', index=False)

### Split de la colonne ID_title

In [22]:
# Supprimer les lignes qui ont des valeurs manquantes dans 'actor_name'
nameBasic_clean = nameBasic_clean.dropna(subset=['profession'])
# Print the first 5 rows
print(nameBasic_clean.head().to_markdown(index=False, numalign="left", stralign="left"))

| ID_name   | actor_name      | profession                         | ID_title                                |
|:----------|:----------------|:-----------------------------------|:----------------------------------------|
| nm0000001 | Fred Astaire    | actor,miscellaneous,producer       | tt0072308,tt0050419,tt0053137,tt0027125 |
| nm0000002 | Lauren Bacall   | actress,soundtrack,archive_footage | tt0037382,tt0075213,tt0117057,tt0038355 |
| nm0000003 | Brigitte Bardot | actress,music_department,producer  | tt0057345,tt0049189,tt0056404,tt0054452 |
| nm0000004 | John Belushi    | actor,writer,music_department      | tt0072562,tt0077975,tt0080455,tt0078723 |
| nm0000005 | Ingmar Bergman  | writer,director,actor              | tt0050986,tt0083922,tt0050976,tt0069467 |


In [23]:
nameBasic_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10590226 entries, 0 to 13578124
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   ID_name     object
 1   actor_name  object
 2   profession  object
 3   ID_title    object
dtypes: object(4)
memory usage: 404.0+ MB


In [24]:
nameBasic_clean.sample()

,ID_name,actor_name,profession,ID_title
6735283,nm1746414,Alain Gagnon,camera_department,"tt0100088,tt0249232,tt0137394"


In [25]:
# Diviser la colonne 'ID title' en plusieurs colonnes
nameBasic_clean['ID_title'] = nameBasic_clean['ID_title'].str.split(',')

# Explode la colonne 'ID_title' pour obtenir plusieurs lignes
nameBasicClean_dummi = nameBasic_clean.explode('ID_title')

# Réinitialise les index pour avoir un DataFrame propre
nameBasicClean_dummi = nameBasicClean_dummi.reset_index(drop=True)

In [26]:
# Calcul de la somme des doublons pour la colonne ID title
doublon = nameBasicClean_dummi.duplicated("ID_title").sum()
print(f'Le nombre de doublons est {doublon}')

Le nombre de doublons est 18453482


In [29]:
nameBasicClean_dummi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20434966 entries, 0 to 20434965
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   ID_name     object
 1   actor_name  object
 2   profession  object
 3   ID_title    object
dtypes: object(4)
memory usage: 623.6+ MB


#### Dichotomisation de la colonne profession

In [30]:
# Split et dichotomisation des valeurs de la colonne 'profession'
profession_dummies = nameBasicClean_dummi['profession'].str.get_dummies(',')

# Concaténer les colonnes dichotomisées avec le DataFrame original
nameBasicClean_dummi = pd.concat([nameBasicClean_dummi, profession_dummies], axis=1)

# Afficher le DataFrame avec la colonne profession dichotomisée
nameBasicClean_dummi.sample()

,ID_name,actor_name,profession,ID_title,accountant,actor,actress,animation_department,archive_footage,archive_sound,...,script_department,set_decorator,sound_department,soundtrack,special_effects,stunts,talent_agent,transportation_department,visual_effects,writer
16109824,nm6123422,Leandro Varela,"director,writer,editor",tt4075174,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [31]:
# On sélectionne uniquement les colonnes qui nous interesse
columns_to_keep = ['ID_name', 'actor_name', 'ID_title', 'actor', 'actress', 'director']

nameBasic_Final = nameBasicClean_dummi[columns_to_keep]
nameBasic_Final.sample()

,ID_name,actor_name,ID_title,actor,actress,director
18964316,nm8678801,B. Mendleson,tt3439754,0,0,0


### 1-Sauvegarder en format CSV

In [32]:
# Une sauvegarde du df avec les valeurs manquantes de la colonne profession non supprimées.
nameBasic_Final.to_csv('nameBasic_Final.csv', index=False)